In [13]:
#Importaciones necesarias
import re  
import json
import nltk
import time
import pickle
import numpy as np

from datetime import datetime
from nltk.stem import SnowballStemmer
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from selenium.webdriver.common.by import By
from tensorflow.keras.models import load_model
from selenium.webdriver.common.keys import Keys   
from tensorflow.keras.layers import Dense, Dropout

In [14]:
ignorar = ['!','¡','¿','?','.']
stemmer = SnowballStemmer('spanish')
model = Sequential()

In [15]:
class Func():
    
    def __init__(self)-> None:
        pass
    
    def Json(self, data, filename):
        self.data = data
        archivo = open(filename,'w')
        json.dump(data, archivo, indent=4)
        
    def Modelo(self, mod):        
        self.model = mod
        
    def Final(self, modelo, tags, palabras):
        self.modelo = modelo
        self.tags = tags
        self.palabras = palabras
        
    def cleanEntrada(self, ent_usuario):
        ent_usuario = nltk.word_tokenize(ent_usuario)
        ent_usuario = [stemmer.stem(w.lower()) for w in ent_usuario if w not in ignorar]
        return ent_usuario

    def convVector(self, ent_usuario):    
        ent_usuario = self.cleanEntrada(ent_usuario)    
        vect_entrada = [0]*len(self.palabras) 
        for palabra in ent_usuario:              
            if palabra in self.palabras:          
                indice = self.palabras.index(palabra)
                vect_entrada[indice] = 1                 
        vect_entrada = np.array(vect_entrada)           
        return vect_entrada

    def getTags(self, vect_entrada, LIMITE = 0):
        vect_salida = self.model.predict(np.array([vect_entrada]))[0]
        vect_salida = [[i,r] for i,r in enumerate(vect_salida) if r > LIMITE]
        vect_salida.sort(key=lambda x: x[1], reverse=True)
        print(vect_salida)    
        list_etiquetas = []    
        for r in vect_salida:   
            list_etiquetas.append({"intent": self.tags[r[0]], "probability": str(r[1])})   
        return list_etiquetas

    def getResponse(self, list_etiquetas):
        etiqueta = list_etiquetas[0]['intent']
        list_data = self.data['intents']
        for dat in list_data:
            if etiqueta == dat['tag']:
                list_respuestas = dat['resp']            
                break
        return list_respuestas

    def chatbotRespuesta(self, ent_usuario):
        vect_entrada = self.convVector(ent_usuario)
        list_etiquetas = self.getTags(vect_entrada, LIMITE = 0)
        respuesta = self.getResponse(list_etiquetas)
        return respuesta


In [16]:
obj = Func()
model = Sequential()

class Chat():
    #Creacion del banco de palabras
    data = { 'intents':[{'tag':'Saludo',
                         'preg':['Hola', 'Holaa', 'Holaaa'],
                         'resp':['Hola soy Shamaris'],}        
    ]}
    obj.Json(data,'intenciones.json')
    obj.Modelo(model)
    
    palabras = []
    tags = []
    documents = []
        
    for intents in data['intents']:
        tags.append(intents['tag'])
        for preguntas in intents['preg']:
            preguntas_res = nltk.word_tokenize(preguntas)
            palabras.extend(preguntas_res) 
            documents.append((preguntas_res, intents['tag']))
    
    palabras = [stemmer.stem(w.lower()) for w in palabras if w not in ignorar]
    
    pickle.dump(palabras, open('palabras.pkl', 'wb'))
    pickle.dump(tags, open('tags.pkl', 'wb'))
    
    entrenamiento = []
    
    for doc in documents:
        inter = doc[0]
        inter = [stemmer.stem(w.lower()) for w in inter if w not in ignorar]
        ent_cod = []
        for pal in palabras:
            if pal in inter:
                ent_cod.append(1)
            else:
                ent_cod.append(0)
        sal_cod = [0]*len(tags)
        indice = tags.index(doc[1])
        sal_cod[indice] = 1
        entrenamiento.append([ent_cod, sal_cod])
    
    entrenamiento = np.array(entrenamiento, dtype = list)
    x_train = list(entrenamiento[:,0])
    y_train = list(entrenamiento[:,1])
    
    model.add(Dense(128, input_shape=(len(x_train[0]),), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation = 'relu')) 
    model.add(Dropout(0.5))
    model.add(Dense(len(y_train[0]),activation = 'softmax'))

    sgd = SGD(learning_rate=0.01,momentum=0.9,nesterov=True) 
    model.compile(loss = "categorical_crossentropy",optimizer = sgd, metrics = ["accuracy"])

    hist = model.fit(np.array(x_train),np.array(y_train),epochs = 300,batch_size = 10,verbose = True)
    model.save("modelo_chatbot.h5", hist)
    
    print("##### MODELO CREADO #####")
    
    modelo = load_model('modelo_chatbot.h5')
    data = json.loads(open('intenciones.json').read())
    palabras = pickle.load(open("palabras.pkl","rb"))
    tags = pickle.load(open("tags.pkl","rb"))
    
    obj.Final(modelo, tags, palabras)

Epoch 1/300
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/300
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/300
1/1 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/300
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/300
1/1 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/300
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/300
1/1 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/300
1/1 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - ac

Epoch 81/300
1/1 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 82/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 83/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 84/300
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 85/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 86/300
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 87/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 88/300
1/1 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 89/300
1/1 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 90/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+0

1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 161/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 162/300
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 163/300
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 164/300
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 165/300
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 166/300
1/1 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 167/300
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 168/300
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 169/300
1/1 [==============================] - 0s 10ms/step - loss: 0.0000

1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 240/300
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 241/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 242/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 243/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 244/300
1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 245/300
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 246/300
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 247/300
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 248/300
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 

In [17]:
#Importacion de selenium para abrir el software automatizado en Edge
from selenium import webdriver
driver = webdriver.Edge()
driver.get("https://web.whatsapp.com/")

In [18]:
def Verificar(usuario):
    try:
        num_mens = usuario.find_element(By.CLASS_NAME,"_1pJ9J").text
        mens_leer = re.findall('\d+' ,num_mens)
        if len(mens_leer) != 0:
            pendiente = True
        else:
            pendiente = False
    except:
        pendiente = False
    return pendiente

def obtenerMensaje(chat):
    mensaje = chat.find_element(By.CLASS_NAME,"_21Ahp").text
    html = chat.get_attribute('outerHTML')
    persona = re.findall(r'data-pre-plain-text=.{1}\[.+](.+?):', html)
    return persona, mensaje

usuario_men = ''

while usuario_men != 'salir':
    print('#', end = " ")
    time.sleep(1)
    conversaciones = driver.find_elements(By.CLASS_NAME, "_1Oe6M") 
    
    for usuario in conversaciones:
        nombres = usuario.find_element(By.CLASS_NAME,"_21S-L")
        no_resp = Verificar(usuario)
        
        if no_resp:
            conversacion = usuario.find_element(By.CLASS_NAME,"vQ0w7")
            conversacion.click()  
            time.sleep(1)        
            chats = driver.find_elements(By.CLASS_NAME,"_27K43")
            
            for chat in chats[-4:]:
                persona, mensaje = obtenerMensaje(chat)
                usuario_men = mensaje.lower()
                
            print("Usuario:",persona, "Mensaje:", usuario_men, end = " ")
            respuesta = driver.find_element(By.CLASS_NAME,"_3Uu1_")            
            bot_resp = obj.chatbotRespuesta(usuario_men)
            
            for res in bot_resp:
                respuesta.send_keys(res, Keys.SHIFT + Keys.ENTER)
            
            respuesta.send_keys(Keys.ENTER)                 
            time.sleep(1)
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()    

# Usuario: [' Luis Leiva'] Mensaje: data = { 'intents':[{'tag':'nabibot_saludo',
                         'preg':['hola', 'hola nabibot', 'holaaa', 'buen día', 'buenas noches', 'buenas tardes'],
                         'resp':['¡hola! soy nabibot, ¿cómo puedo ayudarte?', 'hola, soy nabibot ¿en qué te puedo ayudar?']}
    ]} [[0, 1.0]]
Usuario: [' Ameth'] Mensaje: me lo pueden mandar al privado [[0, 1.0]]
Usuario: [' +507 6767-6184'] Mensaje: ajqjqjajqj [[0, 1.0]]
# # # # # # # 

KeyboardInterrupt: 